# Vitessce Widget Tutorial

# Configure relative to a base_dir

## 1. Import dependencies

We need to import the classes and functions that we will be using from the corresponding packages.

In [1]:
import os
from os.path import join, isfile, isdir
from urllib.request import urlretrieve
from anndata import read_h5ad

from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    AnnDataWrapper,
    BASE_URL_PLACEHOLDER,
)
from vitessce.data_utils import (
    optimize_adata,
    VAR_CHUNK_SIZE,
)

## 2. Define a `base_dir`

We will define a `base_dir` inside which our data will live. We will provide this to `VitessceConfig` in order to construct a configuration that contains URL paths relative to this directory.

In [2]:
BASE_DIR = "data"

## 3. Download the data

For this example, we need to download a dataset from the COVID-19 Cell Atlas https://www.covid19cellatlas.org/index.healthy.html#habib17.

In [3]:
adata_relative_filepath = "habib17.processed.h5ad" # Relative to BASE_DIR
adata_filepath = join(BASE_DIR, adata_relative_filepath)
if not isfile(adata_filepath):
    os.makedirs(BASE_DIR, exist_ok=True)
    urlretrieve('https://covid19.cog.sanger.ac.uk/habib17.processed.h5ad', adata_filepath)

## 4. Load the data

Note: this function may print a `FutureWarning`

In [4]:
adata = read_h5ad(adata_filepath)

/Users/mkeller/software/miniconda3/envs/vitessce-python-notebooks/lib/python3.9/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/Users/mkeller/software/miniconda3/envs/vitessce-python-notebooks/lib/python3.9/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


## 4.1. Preprocess the Data For Visualization

This dataset contains 25,587 genes.  We prepare to visualize the top 50 highly variable genes for the heatmap as ranked by dispersion norm, although one may use any boolean array filter for the heatmap.

In [5]:
top_dispersion = adata.var["dispersions_norm"][
    sorted(
        range(len(adata.var["dispersions_norm"])),
        key=lambda k: adata.var["dispersions_norm"][k],
    )[-51:][0]
]
adata.var["top_highly_variable"] = (
    adata.var["dispersions_norm"] > top_dispersion
)

/var/folders/36/83j6x3ln225bvbpk1_vdzrm00000gn/T/ipykernel_36578/862524975.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  key=lambda k: adata.var["dispersions_norm"][k],
/var/folders/36/83j6x3ln225bvbpk1_vdzrm00000gn/T/ipykernel_36578/862524975.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  top_dispersion = adata.var["dispersions_norm"][


## 4.2 Save the Data to Zarr store

We want to convert the original `h5ad` file to a [Zarr](https://zarr.readthedocs.io/en/stable/) store, which Vitessce is able to load. We can use the `optimize_adata` function to ensure that all arrays and dataframe columns that we intend to use in our visualization are in the optimal format to be loaded by Vitessce. This function will cast arrays to numerical data types that take up less space (as long as the values allow). Note: unused arrays and columns (i.e., not specified in any of the parameters to `optimize_adata`) will not be copied into the new AnnData object.

In [6]:
zarr_relative_filepath = "habib17.processed.zarr" # Relative to BASE_DIR
zarr_filepath = join(BASE_DIR, zarr_relative_filepath)
if not isdir(zarr_filepath):
    adata = optimize_adata(
        adata,
        obs_cols=["CellType"],
        obsm_keys=["X_umap"],
        optimize_X=True,
        var_cols=["top_highly_variable"],
    )
    adata.write_zarr(zarr_filepath, chunks=[adata.shape[0], VAR_CHUNK_SIZE])

## 5. Create the Vitessce widget configuration

Vitessce needs to know which pieces of data we are interested in visualizing, the visualization types we would like to use, and how we want to coordinate (or link) the views.

### 5.1. Instantiate a `VitessceConfig` object

Use the `VitessceConfig` constructor to create an instance. In this case, we want to construct our configuration using local data that is relative to a particular directory, so we provide the `base_dir` parameter.

Note: This `base_dir` parameter is optional. When it is omitted, local data paths are assumed to be relative to the current working directory.

In [7]:
vc = VitessceConfig(schema_version="1.0.15", name='Habib et al', base_dir=BASE_DIR)

### 5.2. Add a dataset to the `VitessceConfig` instance

In Vitessce, a dataset is a container for one file per data type. The `.add_dataset(name)` method on the `vc` instance sets up and returns a new dataset instance.

Then, we can call the dataset's `.add_object(wrapper_object)` method to attach a "data wrapper" instance to our new dataset. For example, the `AnnDataWrapper` helps to configure AnnData Zarr stores for use in the Vitessce configuration.

Dataset wrapper classes may require additional parameters to resolve ambiguities. For instance, `AnnData` objects may store multiple clusterings or cell type annotation columns in the `adata.obs` dataframe. We can use the parameter `obs_set_paths` to tell Vitessce that certain columns of the `obs` dataframe correspond to cell type annotations or cell clusterings.

In [8]:
dataset = vc.add_dataset(name='Brain').add_object(AnnDataWrapper(
        adata_path=zarr_relative_filepath, # Relative to BASE_DIR (because we specified base_dir in the VitessceConfig constructor)
        obs_embedding_paths=["obsm/X_umap"],
        obs_embedding_names=["UMAP"],
        obs_set_paths=["obs/CellType"],
        obs_set_names=["Cell Type"],
        obs_feature_matrix_path="X",
        initial_feature_filter_path="var/top_highly_variable"
    )
)

### 5.3. Add visualizations to the `VitessceConfig` instance

Now that we have added a dataset, we can configure visualizations. The `.add_view` method adds a view (i.e. visualization or controller component) to the configuration.

The `Component` enum class (which we have imported as `cm` here) can be used to fill in the `component_type` parameter.

For convenience, the `SCATTERPLOT` component type takes the extra `mapping` keyword argument, which specifies which embedding should be used for mapping cells to (x,y) points on the plot.

In [9]:
scatterplot = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="UMAP")
cell_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
heatmap = vc.add_view('spatialQuery', dataset=dataset)

### 5.4. Define the visualization layout

The `vc.layout(view_concat)` method allows us to specify how our views will be arranged in the layout grid in the widget. The `|` and `/` characters are magic syntax for `hconcat(v1, v2)` and `vconcat(v1, v2)`, respectively.

In [10]:
vc.layout((scatterplot | cell_sets) / (heatmap | genes));

## 6. Create the widget

The `vc.widget()` method returns the configured widget instance.

In [33]:
PLUGIN_ESM = """
function createPlugins(utilsForPlugins) {
    const {
        React,
        PluginFileType,
        PluginViewType,
        PluginCoordinationType,
        PluginJointFileType,
        z,
        useCoordination,
    } = utilsForPlugins;
    const e = React.createElement;
    function SpatialQueryView(props) {
        const {
            coordinationScopes,
        } = props;
        console.log(coordinationScopes);
        const [{
            myValue,
        }, {
            setMyValue,
        }] = useCoordination(['myValue'], coordinationScopes);

        return e('div', {}, [
            e('p', {}, 'Spatial-Query via PLUGIN_ESM'),
            e('input', { type: 'range', value: myValue, onChange: (e) => setMyValue(parseFloat(e.target.value)), min: 0, max: 100, step: 1 }),
        ]);
    }

    const pluginCoordinationTypes = [
        new PluginCoordinationType('myValue', 0, z.number().nullable()),
    ];

    const pluginViewTypes = [
        new PluginViewType('spatialQuery', SpatialQueryView, ['myValue']),
    ];
    return { pluginViewTypes, pluginCoordinationTypes };
}
export default { createPlugins };
"""

In [34]:
vw = vc.widget(custom_js_url="http://localhost:8000/packages/main/prod/dist/index.min.js", plugin_esm=PLUGIN_ESM, remount_on_uid_change=False)

In [35]:
vw

VitessceWidget(config={'version': '1.0.15', 'name': 'Habib et al', 'description': '', 'datasets': [{'uid': 'A'…

In [18]:
vw = vc.widget(custom_js_url="http://localhost:8000/packages/main/prod/dist/index.min.js", plugin_esm=PLUGIN_ESM, remount_on_uid_change=False)

def handle_change(change):
    new_config = change.new
    if new_config["coordinationSpace"]["myValue"]["A"] >= 50:
        new_config["coordinationSpace"]["obsColorEncoding"]["A"] = "geneSelection"
        vw.config = { **new_config, "uid": "over_50" }
    else:
        new_config["coordinationSpace"]["obsColorEncoding"]["A"] = "cellSetSelection"
        vw.config = { **new_config, "uid": "under_50" }
        

vw.observe(handle_change, names=["config"])

In [ ]:
vw.config["coordinationSpace"]["myValue"]

In [ ]:
vw.config["coordinationSpace"]["obsColorEncoding"]

## 7. Check the URLs in the configuration

We can check that the data URLs in the configuration respected the specified `base_dir`.

In [ ]:
config_dict = vc.to_dict(base_url=BASE_URL_PLACEHOLDER)
config_dict